In [ ]:
!pip install pyctcdecode
!pip install pytorch-transformers

In [ ]:
!pip uninstall transformers
!pip install transformers

Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.8/dist-packages/transformers-4.38.2.dist-info/*
    /usr/local/lib/python3.8/dist-packages/transformers/*
Proceed (Y/n)? 

In [ ]:
!pip install pytorch-transformers

In [ ]:
from transformers import AutoModelForCTC, AutoModelForCausalLM, AutoProcessor
import torchaudio
import torch

In [ ]:
processor = AutoProcessor.from_pretrained("ai4bharat/indicwav2vec-hindi")

In [ ]:
!pip install protobuf

In [ ]:
# DEVICE_ID = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = AutoModelForCTC.from_pretrained("ai4bharat/indicwav2vec-hindi")
!pip install -U tensorflow==2.10 
model = AutoModelForCTC.from_pretrained("ai4bharat/indicwav2vec-hindi")

In [ ]:
audio_file_path = "data_files/firstTest.wav"

# Load the audio file
waveform, original_sample_rate = torchaudio.load(audio_file_path)

# Check the original sample rate
print(f"Original sample rate: {original_sample_rate}")

# Resample the audio to 16,000 Hz if it's not already
if original_sample_rate != 16000:
    resampler = torchaudio.transforms.Resample(original_sample_rate, 16000)
    waveform = resampler(waveform)

# Now, you can proceed with preprocessing and inference as before
input_values = processor(waveform, sampling_rate=16000, return_tensors="pt").input_values

# Perform inference
logits = model(input_values).logits

# Decode the logits to get the transcription
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.decode(predicted_ids[0])

print(transcription)

In [ ]:
import torch
from transformers import AutoModelForCTC, AutoProcessor
import torchaudio.functional as F

# Specify the device
DEVICE_ID = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_ID = "ai4bharat/indicwav2vec-hindi"

# Path to your audio file
audio_file_path = "data_files/firstTest.wav"

# Load the audio file
waveform, original_sample_rate = torchaudio.load(audio_file_path)

# Resample the audio to 16,000 Hz if it's not already
if original_sample_rate != 16000:
    resampled_audio = F.resample(waveform, original_sample_rate, 16000)
else:
    resampled_audio = waveform

# Convert the resampled audio to a NumPy array
resampled_audio_np = resampled_audio.numpy()

# Load the model and processor
model = AutoModelForCTC.from_pretrained(MODEL_ID).to(DEVICE_ID)
processor = AutoProcessor.from_pretrained(MODEL_ID)

# Process the audio
input_values = processor(resampled_audio_np, return_tensors="pt", sampling_rate=16000).input_values

# Perform inference
with torch.no_grad():
    logits = model(input_values.to(DEVICE_ID)).logits.cpu()

# Decode the logits to get the transcription
prediction_ids = torch.argmax(logits, dim=-1)
output_str = processor.batch_decode(prediction_ids)[0]
print(f"Greedy Decoding: {output_str}")


In [ ]:
!/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"

==> Checking for `sudo` access (which may request your password)...
==> This script will install:
/home/linuxbrew/.linuxbrew/bin/brew
/home/linuxbrew/.linuxbrew/share/doc/homebrew
/home/linuxbrew/.linuxbrew/share/man/man1/brew.1
/home/linuxbrew/.linuxbrew/share/zsh/site-functions/_brew
/home/linuxbrew/.linuxbrew/etc/bash_completion.d/brew
/home/linuxbrew/.linuxbrew/Homebrew
==> The following new directories will be created:
/home/linuxbrew/.linuxbrew/bin
/home/linuxbrew/.linuxbrew/etc
/home/linuxbrew/.linuxbrew/include
/home/linuxbrew/.linuxbrew/lib
/home/linuxbrew/.linuxbrew/sbin
/home/linuxbrew/.linuxbrew/share
/home/linuxbrew/.linuxbrew/var
/home/linuxbrew/.linuxbrew/opt
/home/linuxbrew/.linuxbrew/share/zsh
/home/linuxbrew/.linuxbrew/share/zsh/site-functions
/home/linuxbrew/.linuxbrew/var/homebrew
/home/linuxbrew/.linuxbrew/var/homebrew/linked
/home/linuxbrew/.linuxbrew/Cellar
/home/linuxbrew/.linuxbrew/Caskroom
/home/linuxbrew/.linuxbrew/Frameworks

Press RETURN/ENTER to continue 

In [10]:
!/bin/bash -c $sudo apt update && sudo apt install ffmpeg

import torch
# import ffmpeg
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

# def load_audio_with_ffmpeg(audio_file_path):
#     audio, sample_rate = ffmpeg.input(audio_file_path).output("-", format="s16le").run(capture_stdout=True)
#     return audio, sample_rate

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

result = pipe("data_files/firstTest.wav")
print(result["text"])

apt 2.0.9 (amd64)
Usage: apt [options] command

apt is a commandline package manager and provides commands for
searching and managing as well as querying information about packages.
It provides the same functionality as the specialized APT tools,
like apt-get and apt-cache, but enables options more suitable for
interactive use by default.

Most used commands:
  list - list packages based on package names
  search - search in package descriptions
  show - show package details
  install - install packages
  reinstall - reinstall packages
  remove - remove packages
  autoremove - Remove automatically all unused packages
  update - update list of available packages
  upgrade - upgrade the system by installing/upgrading packages
  full-upgrade - upgrade the system by removing/installing/upgrading packages
  edit-sources - edit the source information file
  satisfy - satisfy dependency strings

See apt(8) for more information about the available commands.
Configuration options and syntax is 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


ValueError: ffmpeg was not found but is required to load audio files from filename

In [3]:
import torch
from transformers import AutoModelForCTC, AutoProcessor
import torchaudio

# Specify the device
DEVICE_ID = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_ID = "ai4bharat/indicwav2vec-hindi"

# Path to your audio file
audio_file_path = "data_files/firstTest.wav" # Make sure this path is correct

# Load the audio file
waveform, original_sample_rate = torchaudio.load(audio_file_path)
# Assuming `waveform` is your loaded audio tensor
# Squeeze the tensor to remove the channel dimension
waveform = waveform.squeeze(0)

# Resample the audio to 16,000 Hz if it's not already
if original_sample_rate != 16000:
    resampled_audio = torchaudio.transforms.Resample(original_sample_rate, 16000)(waveform)
else:
    resampled_audio = waveform

# Load the model and processor
model = AutoModelForCTC.from_pretrained(MODEL_ID).to(DEVICE_ID)
processor = AutoProcessor.from_pretrained(MODEL_ID)

# Process the audio
# Note: No need to convert to NumPy array here. Keep it as a PyTorch tensor.
input_values = processor(resampled_audio, return_tensors="pt", sampling_rate=16000).input_values

# Perform inference
with torch.no_grad():
    logits = model(input_values.to(DEVICE_ID)).logits.cpu()
    # logits = logits.reshape(-1, len(processor.tokenizer.vocab))

# Decode the logits to get the transcription
prediction_ids = torch.argmax(logits, dim=-1)
# Wrap prediction_ids in a list since batch_decode expects a list of tensors
# output_str = processor.batch_decode([prediction_ids])[0]
output_str = processor.batch_decode(logits.numpy())
print(f"Greedy Decoding: {output_str}")

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at ai4bharat/indicwav2vec-hindi were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at ai4bharat/indicwav2vec-hindi and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this mode

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Greedy Decoding: Wav2Vec2DecoderWithLMOutput(text=['नमस्कार में डिपार्टमेंट ऑफ एडमिनिस्टे टी रिफॉर्म अन पब्लिक रिगिंस की तरफ से सविता बात करूँ कपिल जी बात करे सर हजील बोलो जी सर आपने डिपार्टमेंट ऑफ पोस्टर से रिलेटेड अपनी शिकायत दर्ज किया था सात अगस्त को ⁇⁇⁇⁇ में सर जीदी जीजिसकी शिकायत संख्या है डबलजीरो डबल टू सिक्स फाई वनसर आपसे जाने'], logit_score=[-41.316065198342194], lm_score=[-41.316065198342194], word_offsets=None)


In [1]:
!pip uninstall -y transformer-engine

Found existing installation: transformer-engine 0.2.0
Uninstalling transformer-engine-0.2.0:
  Successfully uninstalled transformer-engine-0.2.0


In [ ]:
# Datasets :

# For Hindi -> ENglish worded translation
# https://huggingface.co/datasets/cfilt/iitb-english-hindi
    
# For ASR:
# 1. https://huggingface.co/datasets/TheAIchemist13/hindi_asr_dataset
# 2. https://huggingface.co/datasets/TheAIchemist13/hindi_asr_dataset_2

In [1]:
from datasets import load_dataset

dataset1 = load_dataset("TheAIchemist13/hindi_asr_dataset")
dataset2 = load_dataset("TheAIchemist13/hindi_asr_dataset_2")

In [2]:
from datasets import load_dataset, DatasetDict, concatenate_datasets

combined_dataset1 = DatasetDict()
combined_dataset2 = DatasetDict()
combined_dataset = DatasetDict()

combined_dataset1["train"] = load_dataset("TheAIchemist13/hindi_asr_dataset", "default", split="train", use_auth_token=True)
combined_dataset1["test"] = load_dataset("TheAIchemist13/hindi_asr_dataset", "default", split="test", use_auth_token=True)

combined_dataset2["train"] = load_dataset("TheAIchemist13/hindi_asr_dataset_2", "default", split="train", use_auth_token=True)
combined_dataset2["test"] = load_dataset("TheAIchemist13/hindi_asr_dataset_2", "default", split="test", use_auth_token=True)

combined_dataset["train"] = concatenate_datasets([combined_dataset1["train"], combined_dataset2["train"]])
combined_dataset["test"] = concatenate_datasets([combined_dataset1["test"], combined_dataset2["test"]])
combined_dataset

/usr/local/lib/python3.8/dist-packages/datasets/load.py:2508: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['audio', 'transcriptions'],
        num_rows: 255
    })
    test: Dataset({
        features: ['audio', 'transcriptions'],
        num_rows: 95
    })
})

In [3]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-large-v3")

In [4]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-large-v3", language="Hindi", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-large-v3", language="Hindi", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
from datasets import Audio

# for split_name in combined_dataset.keys():
#     combined_dataset[split_name] = combined_dataset[split_name].cast_column("audio", Audio(sampling_rate=16000))
combined_dataset = combined_dataset.cast_column("audio", Audio(sampling_rate=16000))

In [7]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["transcriptions"]).input_ids
    return batch

In [8]:
print(combined_dataset["train"][0])

{'audio': {'path': '1.wav', 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 16000}, 'transcriptions': 'बीकीपर अक्सर अपनी कॉलोनियों के स्वास्थ्य को सुनिश्चित करने के लिए अमेरिकन फाउलब्रूड परीक्षणों का उपयोग करते हैं।'}


In [9]:
common_voice = combined_dataset.map(prepare_dataset, remove_columns=combined_dataset.column_names["train"], num_proc=5)

In [10]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [11]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [12]:
import evaluate

metric = evaluate.load("wer")

In [13]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [14]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3")

In [15]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [16]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-large-v3",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [17]:
import accelerate, transformers

print(transformers.__version__)
accelerate.__version__

4.38.2


'0.28.0'

In [19]:
from huggingface_hub import notebook_login

notebook_login()

In [20]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
#     tokenizer=tokenizer
    tokenizer=processor.feature_extractor,
)

/usr/local/lib/python3.8/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [21]:
processor.save_pretrained(training_args.output_dir)

[]

In [22]:
import torch
torch.cuda.empty_cache()

In [23]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   6016 MiB |   6016 MiB |   6016 MiB |      0 B   |\n|       from large pool |   6012 MiB |   6012 MiB |   6012 MiB |      0 B   |\n|       from small pool |      4 MiB |      4 MiB |      4 MiB |      0 B   |\n|---------------------------------------------------------------------------|\n| Active memory         |   6016 MiB |   6016 MiB |   6016 MiB |      0 B   |\n|       from large pool |   6012 MiB |   6012 MiB |

In [1]:
trainer.train()

NameError: name 'trainer' is not defined

In [ ]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: hi, split: test",
    "language": "hi",
    "model_name": "Whisper Small Hi - Sanchit Gandhi",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [ ]:
trainer.push_to_hub(**kwargs)

In [ ]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="sanchit-gandhi/whisper-small-hi")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

iface = gr.Interface(
    fn=transcribe, 
    inputs=gr.Audio(source="microphone", type="filepath"), 
    outputs="text",
    title="Whisper Small Hindi",
    description="Realtime demo for Hindi speech recognition using a fine-tuned Whisper small model.",
)

iface.launch()